Passpunkte

| 0  | 1  | 2  | 3  |
|----|----|----|----|
| 4  | 5  | 6  | 7  |
| 8  | 9 | 10 | 11 |
| 12 | 13 | 14 | 15 |

In [45]:
passpunkte = [
    [0, 0, 0],
    [1, 0, 0],
    [2, 0, 0],
    [3, 0, 0],
    [0, 1, 0],
    [1, 1, 0],
    [2, 1, 0],
    [3, 1, 0],
    [0, 2, 0],
    [1, 2, 0],
    [2, 2, 0],
    [3, 2, 0],
    [0, 3, 0],
    [1, 3, 0],
    [2, 3, 0],
    [3, 3, 0]
]


In [46]:
# kamera, bildnr, passpunkt, bildkoordinatenx, y
messungen = [
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 2, 2, 0],
    [0, 0, 4, 0, 1],
    [0, 0, 5, 1, 1],
    [0, 0, 6, 2, 1],
    [0, 0, 8, 0, 2],
    [0, 0, 9, 1, 2],
    [0, 0, 10, 2, 2],
    [0, 1, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 1, 3, 2, 0],
    [0, 1, 5, 0, 1],
    [0, 1, 6, 1, 1],
    [0, 1, 7, 2, 1],
    [0, 1, 9, 0, 2],
    [0, 1, 10, 1, 2],
    [0, 1, 11, 2, 2],
    [0, 2, 4, 0, 0],
    [0, 2, 5, 1, 0],
    [0, 2, 6, 2, 0],
    [0, 2, 8, 0, 1],
    [0, 2, 9, 1, 1],
    [0, 2, 10, 2, 1],
    [0, 2, 12, 0, 2],
    [0, 2, 13, 1, 2],
    [0, 2, 14, 2, 2],
    [0, 3, 5, 0, 0],
    [0, 3, 6, 1, 0],
    [0, 3, 7, 2, 0],
    [0, 3, 9, 0, 1],
    [0, 3, 10, 1, 1],
    [0, 3, 11, 2, 1],
    [0, 3, 13, 0, 2],
    [0, 3, 14, 1, 2],
    [0, 3, 15, 2, 2],
    [0, 4, 0, 0, 0],
    [0, 4, 1, 1, 0],
    [0, 4, 2, 2, 0],
    [0, 4, 3, 3, 0],
    [0, 4, 4, 0, 1],
    [0, 4, 5, 1, 1],
    [0, 4, 6, 2, 1],
    [0, 4, 7, 3, 1],
    [0, 4, 8, 0, 2],
    [0, 4, 9, 1, 2],
    [0, 4, 10, 2, 2],
    [0, 4, 11, 3, 2],
    [0, 4, 12, 0, 3],
    [0, 4, 13, 1, 3],
    [0, 4, 14, 2, 3],
    [0, 4, 15, 3, 3]
]


In [47]:
# x,y,z, height, width
bild = [
    [1, 1, 1, 2,2],
    [2, 1, 1, 2, 2],
    [1, 2, 1, 2, 2],
    [2, 2, 1, 2, 2],
    [1.5, 1.5, 1.5, 3, 3]
]


In [48]:
import time
import numpy as np
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix


In [49]:
camera_indices = np.array([i[0] for i in messungen])
image_indices = np.array([i[1] for i in messungen])
point_indices = np.array([i[2] for i in messungen])
points_2d = np.array([[i[3]/bild[i[1]][3]-0.5, i[4]/bild[i[1]][4]-0.5]
                     for i in messungen])
points_3d = np.array(passpunkte)
image_params = np.array([[1, 1, 0, i[0], i[1], i[2]] for i in bild])
camera_params = np.array([[0.5, 0, 0]])


In [50]:
n_cameras = camera_params.shape[0]
n_images = image_params.shape[0]
n_points = points_3d.shape[0]

n = n_cameras * 3 + 6 * n_images + 3 * n_points
m = 2 * points_2d.shape[0]

print("n_cameras: {}".format(n_cameras))
print("n_images: {}".format(n_images))
print("n_points: {}".format(n_points))
print("Total number of parameters: {}".format(n))
print("Total number of residuals: {}".format(m))

n_cameras: 1
n_images: 5
n_points: 16
Total number of parameters: 81
Total number of residuals: 104


In [51]:
def rotate(points, rot_vecs):
    """Rotate points by given rotation vectors.
    
    Rodrigues' rotation formula is used.
    """
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v


In [52]:
def project(points, camera_params, image_params):
    """Convert 3-D points to 2-D by projecting onto images."""
    points_proj = rotate(points, image_params[:, :3])
    points_proj += image_params[:, 3:6]
    points_proj = -points_proj[:, :2] / points_proj[:, 2, np.newaxis]
    f = camera_params[:, 0]
    k1 = camera_params[:, 1]
    k2 = camera_params[:, 2 ]
    n = np.sum(points_proj**2, axis=1)
    r = 1 + k1 * n + k2 * n**2
    points_proj *= (r * f)[:, np.newaxis]
    return points_proj


In [53]:
def fun(params, n_cameras, n_images, n_points, camera_indices, images_indices, point_indices, points_2d):
    """Compute residuals.

    `params` contains camera parameters and 3-D coordinates.
    """
    camera_params = params[:n_cameras * 3].reshape((n_cameras, 3))
    image_params = params[n_cameras * 3: n_cameras *
                          3+n_images*6].reshape((n_images, 6))
    points_3d = params[n_cameras*3+n_images*6:].reshape((n_points, 3))
    points_proj = project(
        points_3d[point_indices], camera_params[camera_indices], image_params[image_indices])
    return (points_proj - points_2d).ravel()


In [54]:
def bundle_adjustment_sparsity(n_cameras, n_images, n_points, camera_indices, image_indices, point_indices):
    m = image_indices.size * 2
    n = n_cameras * 3 + n_images * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)

    for s in range(3):
        A[2 * i, camera_indices * 3 + s] = 1
        A[2 * i + 1, camera_indices * 3 + s] = 1

    for s in range(6):
        A[2 * i, n_cameras * 3 + image_indices * 6 + s] = 1
        A[2 * i + 1, n_cameras * 3+ image_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, n_cameras * 3 + n_images * 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, n_cameras * 3 + n_images * 6 + point_indices * 3 + s] = 1

    return A


In [55]:
x0 = np.hstack((camera_params.ravel(), image_params.ravel(), points_3d.ravel()))
f0 = fun(x0, n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d)
x0.reshape(27,3)

array([[0.5, 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 1. , 1. ],
       [0. , 1. , 0. ],
       [2. , 1. , 1. ],
       [0. , 1. , 0. ],
       [1. , 2. , 1. ],
       [0. , 1. , 0. ],
       [2. , 2. , 1. ],
       [0. , 1. , 0. ],
       [1.5, 1.5, 1.5],
       [0. , 0. , 0. ],
       [1. , 0. , 0. ],
       [2. , 0. , 0. ],
       [3. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 1. , 0. ],
       [2. , 1. , 0. ],
       [3. , 1. , 0. ],
       [0. , 2. , 0. ],
       [1. , 2. , 0. ],
       [2. , 2. , 0. ],
       [3. , 2. , 0. ],
       [0. , 3. , 0. ],
       [1. , 3. , 0. ],
       [2. , 3. , 0. ],
       [3. , 3. , 0. ]])

In [56]:

A = bundle_adjustment_sparsity(
    n_cameras, n_images, n_points, camera_indices, image_indices,  point_indices)
t0 = time.time()
res = least_squares(fun, x0, jac_sparsity=A, verbose=2, x_scale='jac', ftol=1e-4, method='trf',
                    args=(n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d))
t1 = time.time()


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.2258e+03                                    1.48e+09    
       1              2         9.8784e+01      1.13e+03       7.72e+00       4.08e+06    
       2              3         6.7433e+01      3.14e+01       8.30e+00       2.78e+07    
       3              4         5.2019e+01      1.54e+01       9.30e+00       4.14e+06    
       4              5         9.9303e+00      4.21e+01       8.41e-01       2.97e+04    
       5              6         2.3773e+00      7.55e+00       8.96e-01       8.41e+01    
       6              8         1.8397e+00      5.38e-01       3.68e-01       7.97e+00    
       7              9         1.4341e+00      4.06e-01       7.72e-01       2.87e+01    
       8             10         1.0902e+00      3.44e-01       2.39e+00       1.23e+02    
       9             12         9.9467e-01      9.56e-02       3.50e-01       3.38e+01    

In [57]:
x0
np.array(res.x.round(1)).reshape(27,3)

array([[  1.3,  -0. ,  -0. ],
       [  0. ,   0.7,   0.2],
       [ -0.8,  -0.1,   0.9],
       [  0.1,   1. ,   0.2],
       [ -0.5,  -0.1,   1.1],
       [ -0.1,   0.7,  -0.1],
       [ -0.8,   0.3,   0.9],
       [ -0.1,   1. ,  -0.1],
       [ -0.5,   0.3,   1.1],
       [ -0. ,   0.9,   0.1],
       [ -0.6,   0.1,   1.1],
       [  6.6,  -4.3, -11.3],
       [  1.2,  -0.1,  -0.2],
       [  2.3,  -0.7,  -0.8],
       [ 12.1,  -5.2,  -3. ],
       [  1.2,  -0.1,  -0.2],
       [  1.2,  -0.1,  -0.2],
       [  2.5,  -0.4,  -0.9],
       [ 12.4,  -2.1,  -2.7],
       [  1.2,  -0.1,  -0.2],
       [  1.2,  -0.1,  -0.2],
       [  2.5,   0.1,  -0.9],
       [ 11.4,   1.3,  -2.3],
       [  1.2,  -0.1,  -0.2],
       [  1.2,  -0.1,  -0.2],
       [  2.2,   0.4,  -0.7],
       [  9.4,   3.2,  -2. ]])